In [ ]:
!pip install transformers

In [ ]:
import torch
import pandas as pd
from transformers import ElectraForSequenceClassification, ElectraTokenizerFast

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/propaganda-on-social-media-v1.2.csv', index_col=None)

# df["label"] = df.category.apply(lambda cat: 1 if cat == 'Discrediting Pages' else 2 if cat == 'Ideologized Propaganda ' else 3 if cat == 'False Media Pages' else 4 if cat == 'False Supporting Pages' else 5 if cat == 'Government Propaganda' else 0)
df["label"] = df.category.apply(lambda cat: 1 if cat == 'Ideologized Propaganda ' else 0)

df_ = pd.concat([df[df.label == 1], df[df.label == 0].sample(15000)])

sentences = df_.post.values
labels = df_.label.values

In [ ]:
tokenizer = ElectraTokenizerFast.from_pretrained("jnz/electra-ka-fake-news-tagging", do_lower_case=True)

In [ ]:
config = {
    'learning_rate': 5e-5,
    'batch_size': 16,
    'epochs':3
}

# Tonekize

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  დიდი ხნის შემდეგ ვბრუნდებით მაგრამ თქვენთვის ცუდი ამბები გვაქვს რადგან გვინდა გაჩვენოთ ადამიანები რომლებსაც ჰგონიათ რომ საქართველოს ისტორია სულ რაღაც ცხრა წელს ითვლის
Token IDs: tensor([62001,  3804, 29338,  8389, 62000,   804,  3293,   982, 14902,  1390,
          540, 38480,   307, 24418,  6849,  6656,    68,     3, 62000, 23985,
        60435,  1770, 29778,   794,  5787, 33140,  4473, 10657, 32229, 36035,
          228,   428, 10941, 62003, 61999, 61999, 61999, 61999, 61999, 61999,
        61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999,
        61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999,
        61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999,
        61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999,
        61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999,
        61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999, 61999,
        61999, 61999, 61999, 61

# Train-Test Split

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

26,187 training samples
2,910 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# import wandb
# WANDB PARAMETER
def ret_dataloader():
    batch_size = config["batch_size"]
    print('batch_size = ', batch_size)
    train_dataloader = DataLoader(
                train_dataset,  # The training samples.
                sampler = RandomSampler(train_dataset), # Select batches randomly
                batch_size = batch_size # Trains with this batch size.
            )

    validation_dataloader = DataLoader(
                val_dataset, # The validation samples.
                sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                batch_size = batch_size # Evaluate with this batch size.
            )
    return train_dataloader,validation_dataloader

# Load Pre-trained BERT model

In [ ]:
from transformers import ElectraForSequenceClassification, AdamW, BertConfig

def ret_model():
    model = ElectraForSequenceClassification.from_pretrained("jnz/electra-ka-fake-news-tagging", 
                                                             num_labels = 2)
                                                            #  output_attentions = False, 
                                                            #  output_hidden_states = False )
    return model

In [ ]:
def ret_optim(model):
    print('Learning_rate = ', config["learning_rate"] )
    optimizer = AdamW(model.parameters(),
                      lr = config["learning_rate"], 
                      eps = 1e-8 
                    )
    return optimizer

In [ ]:
from transformers import get_linear_schedule_with_warmup

def ret_scheduler(train_dataloader,optimizer):
    epochs = config["epochs"]
    print('epochs =>', epochs)
    # Total number of training steps is [number of batches] x [number of epochs]. 
    # (Note that this is not the same as the number of training samples).
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    return scheduler

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
torch.multiprocessing.set_start_method('spawn', force=True)

#Train

In [ ]:
from tqdm import tqdm_notebook as tqdm
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = ret_model()
model.to(device)
#wandb.init(config=sweep_defaults)
train_dataloader,validation_dataloader = ret_dataloader()
optimizer = ret_optim(model)
scheduler = ret_scheduler(train_dataloader,optimizer)

#print("config ",wandb.config.learning_rate, "\n",wandb.config)
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
#torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = config["epochs"]
# For each epoch...
for epoch_i in tqdm(range(0, epochs)):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels)
        # wandb.log({'train_batch_loss':loss.item()})
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    # wandb.log({'avg_train_loss':avg_train_loss})

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    # wandb.log({'val_accuracy':avg_val_accuracy,'avg_val_loss':avg_val_loss})
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

cuda


Some weights of the model checkpoint at /content/electra/ were not used when initializing ElectraForSequenceClassification: ['generator.embeddings.word_embeddings.weight', 'generator.embeddings.position_embeddings.weight', 'generator.embeddings.token_type_embeddings.weight', 'generator.embeddings.LayerNorm.weight', 'generator.embeddings.LayerNorm.bias', 'generator.encoder.layer.0.attention.self.query.weight', 'generator.encoder.layer.0.attention.self.query.bias', 'generator.encoder.layer.0.attention.self.key.weight', 'generator.encoder.layer.0.attention.self.key.bias', 'generator.encoder.layer.0.attention.self.value.weight', 'generator.encoder.layer.0.attention.self.value.bias', 'generator.encoder.layer.0.attention.output.dense.weight', 'generator.encoder.layer.0.attention.output.dense.bias', 'generator.encoder.layer.0.attention.output.LayerNorm.weight', 'generator.encoder.layer.0.attention.output.LayerNorm.bias', 'generator.encoder.layer.0.intermediate.dense.weight', 'generator.encode

batch_size =  16
Learning_rate =  5e-05
epochs => 3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



======== Epoch 1 / 3 ========
Training...
  Batch    40  of  1,637.    Elapsed: 0:00:04.
  Batch    80  of  1,637.    Elapsed: 0:00:07.
  Batch   120  of  1,637.    Elapsed: 0:00:10.
  Batch   160  of  1,637.    Elapsed: 0:00:14.
  Batch   200  of  1,637.    Elapsed: 0:00:17.
  Batch   240  of  1,637.    Elapsed: 0:00:20.
  Batch   280  of  1,637.    Elapsed: 0:00:24.
  Batch   320  of  1,637.    Elapsed: 0:00:27.
  Batch   360  of  1,637.    Elapsed: 0:00:31.
  Batch   400  of  1,637.    Elapsed: 0:00:34.
  Batch   440  of  1,637.    Elapsed: 0:00:38.
  Batch   480  of  1,637.    Elapsed: 0:00:41.
  Batch   520  of  1,637.    Elapsed: 0:00:45.
  Batch   560  of  1,637.    Elapsed: 0:00:48.
  Batch   600  of  1,637.    Elapsed: 0:00:52.
  Batch   640  of  1,637.    Elapsed: 0:00:55.
  Batch   680  of  1,637.    Elapsed: 0:00:59.
  Batch   720  of  1,637.    Elapsed: 0:01:02.
  Batch   760  of  1,637.    Elapsed: 0:01:06.
  Batch   800  of  1,637.    Elapsed: 0:01:09.
  Batch   840  of

In [ ]:
model.save_pretrained("/content/electra")

# Predict

In [ ]:
# df_ = pd.concat([df[df.label == 1], df[df.label == 0].sample(15000)])
for i in range(10):
    post = df[df.label == 1].sample(1).iloc[0].post
    inputs = tokenizer(post, return_tensors="pt")
    inputs.to('cuda')
    res = model(**inputs)
    print(1, res[0].tolist()[0], post)
for i in range(10):
    post = df[df.label == 0].sample(1).iloc[0].post
    inputs = tokenizer(post, return_tensors="pt")
    inputs.to('cuda')
    res = model(**inputs)
    print(0, res[0].tolist()[0], post)


1 [-0.3441304862499237, 0.3961566388607025] სერგის გატეხვით და დასჯით ჩვენი შვილების დაშინებაც სურთ
1 [-1.6734694242477417, 1.8392516374588013] ტროპარიწმიდისაგან ხატისა შენისა ჰოი დედუფალო ღვთისმშობელო სიმრთელენი და კურნებანი უხვად მოეცემიან ყოველთა სარწმუნოებითა და სიყვარულით მისდა მოლტოლვილთა ამისთვის ჩემსაცა უძლურებასა მოხედე და შეიწყალე სული ჩემი სახიერო ხოლო ხორცით განმკურნე მადლითა შენითა ყოვლადწმიდაო
1 [0.9612675309181213, -1.0451594591140747] ნაცების მდგომარეობა უფრო დამძიმდა
1 [-1.6281352043151855, 1.7903770208358765] საქართველოში ყოველდღე დღესაცწიწამურთან კვლავ ტყვიას ისვრიანმოღრუბრულია ისევე ზეცაცდა აქ ყოველდღე კლავენ ილიას
1 [-1.8438581228256226, 2.0151760578155518] დადუმებული ქართველობა updated their status
1 [-1.8136552572250366, 1.9816479682922363] საქართველო უპირველეს ყოვლისა updated their status
1 [-0.014555711299180984, 0.04023725911974907] იტალიის შს მინისტრი რიჩარდ გირს თუ ასე ღელავს მიგრანტებზე შეუძლია საკუთარი კერძო თვითმფრინავით ჰოლივუდში წაიყვანოს და თავის ვილებ